In [2]:
import os
from dotenv import load_dotenv

from autogen_ext.models.openai import AzureOpenAIChatCompletionClient  
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

Create the client

In [3]:
load_dotenv()
model = os.environ["AZURE_OPENAI_MODEL"]
deployment = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]
endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_version  = os.environ.get("AZURE_OPENAI_API_VERSION", "2025-01-01-preview")

client = AzureOpenAIChatCompletionClient(
    model=model,
    endpoint=endpoint,
    azure_deployment=deployment,
    api_version=api_version,
    api_key=api_key
)

In [5]:
# Test client
result = await client.create([
    UserMessage(content="What is the capital of France?", source="user") 
])
print(result.content)

The capital of France is Paris.


Defining the Agent

In [7]:
agent = AssistantAgent(
    name="Assistant",
    model_client=client,
    system_message="You are a travel agent that plans great vacations"
)

In [10]:
# Test agent
res = await agent.run(task="What is the capital of France ?")
for message in res.messages:
    if message.source.lower() == "assistant":
        print(message.content)

The capital of France is Paris.


Run the agent

In [5]:
from IPython.display import display, HTML


async def assistant_run():
    user_query = "Plan me a great sunny vacation"

    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    display(HTML(html_output))

await assistant_run()